## Wet Test

In [5]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
import matplotlib.pyplot as plt
import operator
from functools import reduce, partial
from timeit import default_timer
import pickle
import matplotlib.pyplot as plt
from util import UnitGaussianNormalizer, LpLoss
from neuralop.models import SFNO

torch.manual_seed(45)
np.random.seed(45)

TRAIN_PATH = '/wanghaixin/Cell_Division/wet_636_train.pt'
TEST_PATH = '/wanghaixin/Cell_Division/wet_160_val.pt'
batch_size = 64

file_train = torch.load(TRAIN_PATH)
file_test = torch.load(TEST_PATH)

file_normalizer = UnitGaussianNormalizer(file_train)
file_train = file_normalizer.encode(file_train)
file_test = file_normalizer.encode(file_test)
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(file_test), batch_size=batch_size, shuffle=False)

k = 3
num_timesteps = 20
N = 1
model = SFNO(n_modes=(16, 16), hidden_channels=64,
                in_channels=k, out_channels=k).cuda()
state_dict = torch.load('/wanghaixin/Cell_Division/ckpt/Best_SFNO_wet_{}.pth'.format(int(k)))

model.load_state_dict(state_dict)

with torch.no_grad():
    test_mse = 0
    test_l2_full = 0
    for inputs in test_loader:
        inputs = inputs[0].cuda() # [bs, 32, 32, 20+2]
        
        num_timesteps = inputs.shape[1]
        seq_loss = 0
        seq_l2_loss = 0
        for t in range(0, num_timesteps - 2*k +1, N):
            x = inputs[:,t:t+k, :, :]
            y = inputs[:, t+k:t+2*k, :, :]

            out = model(x)
            out = out.reshape(x.shape[0],-1)
            y = y.reshape(y.shape[0],-1)
            loss = F.mse_loss(out, y, reduction='mean')
            seq_loss += loss.item()
            l2_full = (torch.norm((out - y).flatten(start_dim=1), dim=1) / torch.norm(y.flatten(start_dim=1), dim=1)).mean(0)
            seq_l2_loss += l2_full.item()
        seq_loss/=len(range(0, num_timesteps - 2*k + 1, N))
        seq_l2_loss/=len(range(0, num_timesteps - 2*k + 1, N))
        test_mse +=seq_loss
        test_l2_full += seq_l2_loss
                    
    test_mse /= len(test_loader)
    test_l2_full /= len(test_loader)
        
    t2 = default_timer()
    print('MSE', test_mse, 'L2', test_l2_full)

MSE 0.05305360290739272 L2 0.3147543509801229


## Tension

In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
import matplotlib.pyplot as plt
import os
try:
    from neuralop.models import TFNO
except:
    os.system('pip install neuraloperator torch_harmonics wandb')
from neuralop.models import TFNO
from functools import reduce, partial
from timeit import default_timer
import pickle
import matplotlib.pyplot as plt
from util import UnitGaussianNormalizer, LpLoss, rfft, irfft
import argparse

def load_data(file):
    data_load_file = []
    file_1 = open(file, "rb")
    data_load_file = pickle.load(file_1)
    return data_load_file

TRAIN_PATH = '/wanghaixin/Cell_Division/train_data_tension.pickle'
TEST_PATH = '/wanghaixin/Cell_Division/test_data_tension.pickle'

file_train = load_data(TRAIN_PATH)
file_test = load_data(TEST_PATH)

file_normalizer = UnitGaussianNormalizer(file_train)
file_train = file_normalizer.encode(file_train)
file_test = file_normalizer.encode(file_test)

test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(file_test), batch_size=batch_size, shuffle=False)

k = 1
num_timesteps = 20
N = 1
model = SFNO(n_modes=(16, 16), hidden_channels=64,
                in_channels=k, out_channels=k).cuda()
state_dict = torch.load('/wanghaixin/Cell_Division/ckpt/Best_SFNO_tension_{}.pth'.format(int(k)))

model.load_state_dict(state_dict)

with torch.no_grad():
    test_mse = 0
    test_l2_full = 0
    for inputs in test_loader:
        inputs = inputs[0].cuda() # [bs, 32, 32, 20+2]
        
        num_timesteps = inputs.shape[1]
        seq_loss = 0
        seq_l2_loss = 0
        for t in range(0, num_timesteps - 2*k +1, N):
            x = inputs[:,t:t+k, :, :]
            y = inputs[:, t+k:t+2*k, :, :]

            out = model(x)
            out = out.reshape(x.shape[0],-1)
            y = y.reshape(y.shape[0],-1)
            loss = F.mse_loss(out, y, reduction='mean')
            seq_loss += loss.item()
            l2_full = (torch.norm((out - y).flatten(start_dim=1), dim=1) / torch.norm(y.flatten(start_dim=1), dim=1)).mean(0)
            seq_l2_loss += l2_full.item()
        seq_loss/=len(range(0, num_timesteps - 2*k + 1, N))
        seq_l2_loss/=len(range(0, num_timesteps - 2*k + 1, N))
        test_mse +=seq_loss
        test_l2_full += seq_l2_loss
                    
    test_mse /= len(test_loader)
    test_l2_full /= len(test_loader)
        
    t2 = default_timer()
    print('MSE', test_mse, 'L2', test_l2_full)

MSE 0.07914545504670395 L2 0.6616640357594741
